In [1]:
import pandas
import geopandas

import os, sys
sys.path.append(os.path.abspath(''))

from functions import get_raw_path, get_data_path, store_context_data, open_geopandas_csv,\
                      add_neighbourhood_column, print_df_columns, open_geopandas_csv, geom_dict_to_shape,\
                      get_neighbourhoods, get_gdf_using_geom_dict
import functions

import re

import numpy as np

In [ ]:
raise Exception("Do not run all cells! -> This will overwrite the pre-reformated data!")

# Traffic Volumes

In [ ]:
FILE_NAME = 'raw-data-2010-2019.csv'

STORE_NAME = 'Traffic Volumes.csv'

df = pandas.read_csv(get_raw_path(FILE_NAME))

# df.head()

The coordinates are stored in columns 'lng' and 'lat'

In [ ]:
gdf = geopandas.GeoDataFrame(df.drop(['lng', 'lat'], axis=1), geometry=geopandas.points_from_xy(df['lng'], df['lat']))
gdf.crs = functions.CRS

In [ ]:
gdf = add_neighbourhood_column(gdf)

In [ ]:
gdf.head()

In [ ]:
gdf['Neighbourhood'].isna().sum()

In [ ]:
store_context_data(gdf, STORE_NAME)

In [ ]:
gdf = open_geopandas_csv(get_data_path(STORE_NAME))

gdf.head()

# gdf.loc[:100].explore()

# Centrelines
Because this consists of lines, we use 'intersects' predicate over 'within' to match it.

Therefore an additional columns are added:

- NeighbourhoodPortion (the portion of the MultiLineString's full length that lies within the neighbourhood)

In [ ]:
FILE_NAME  = "Centreline - Version 2 - 4326.csv"
STORE_NAME = "Centrelines.csv"

df = pandas.read_csv(get_raw_path(FILE_NAME))

df.head()

In [ ]:
gdf = get_gdf_using_geom_dict(df)

In [ ]:
## Get the neighbourhood dataset
neighbourhood = get_neighbourhoods()

print("Joining...")
## Merge the gdf with the neighbourhood stuff
joined = geopandas.sjoin(gdf.to_crs(functions.CRS), neighbourhood.to_crs(functions.CRS), how='inner', predicate='within').to_crs(epsg = functions.EPSG)

print("Filtering rows...")
## Filter out any un-matched columns
gdf = gdf.loc[joined.index]

print("Calculating NeighbourhoodPortions")
## Add the 'NeighbourhoodPortion' column
geometry_right = neighbourhood.to_crs(epsg=functions.EPSG).loc[joined['index_right']].set_index(joined.index)
numerator = np.array(joined.geometry.intersection(geometry_right).length)
denomenator = np.array(joined.geometry.length)
gdf['NeighbourhoodPortion'] =  numerator / denomenator 

print("Assigning Neighbourhood")
## Add the Neighbourhood column
gdf['Neighbourhood'] = joined['index_right']

## Drop where they are right on the border, and not inside... so NeighbourhoodPortion is 0
gdf = gdf.loc[gdf['NeighbourhoodPortion'] != 0]

In [ ]:
store_context_data(gdf, STORE_NAME)

# Intersections
Some intersections lie on the border between neighbourhoods, so an additional field is added:
- SingleNeighbourhood: 1 if point lies entirely within a single neighbourhood

In [31]:
FILE_NAME  = "Centreline Intersection - 4326.csv"
STORE_NAME = "Intersections.csv"

df = pandas.read_csv(get_raw_path(FILE_NAME))

df.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION_LEVEL,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,geometry
0,1,13470264,NaN,NaN,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,1,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
1,2,13470193,NaN,NaN,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,4,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
2,3,13470188,NaN,NaN,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,5,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
3,4,13470203,NaN,NaN,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,7,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
4,5,13470228,NaN,NaN,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,9,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."


In [32]:
gdf = get_gdf_using_geom_dict(df)

# gdf.head()

Processing geometry column...
Creating GeoDataFrame


In [33]:
gdf_fully_within = add_neighbourhood_column(gdf)
gdf_fully_within['SingleNeighbourhood'] = np.ones(len(gdf_fully_within))

print("Searching for missing...")
missing_indexes = gdf.index.difference(gdf_fully_within.index)
missing = gdf.loc[missing_indexes]
print(len(missing))

gdf_bordering = add_neighbourhood_column(missing, predicate='intersects')
gdf_bordering['SingleNeighbourhood'] = np.zeros(len(gdf_bordering))
print("Bordering: ", len(gdf_bordering))

print("Searching for final missing ones...")
missing_indexes = missing.index.difference(gdf_bordering.index)
missing = missing.loc[missing_indexes]
print(len(missing))

# missing.head()
print("Combining now...")
gdf = geopandas.GeoDataFrame(pandas.concat([
    gdf_fully_within,
    gdf_bordering
])).set_geometry('geometry').to_crs(functions.CRS)

Joining now using: 'how=inner', 'predicate=within'
Searching for missing...
5268
Joining now using: 'how=inner', 'predicate=intersects'
Bordering:  10175
Searching for final missing ones...
154
Combining now...


In [34]:
# missing.explore()

In [35]:
store_context_data(gdf, STORE_NAME)